In [17]:
import os
import torch
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from PIL import Image
from sklearn.model_selection import train_test_split
from google.colab import drive

In [18]:
# Use this to connect to Google Drive in Google Colab
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
# Use this to unzip file in Google Colab
!unzip drive/MyDrive/data

Archive:  drive/MyDrive/data.zip
replace __MACOSX/._data? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [22]:
class image_data_set(torch.utils.data.Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
  
    def __len__(self):
        return len(self.data)
  
    def __getitem__(self, index):
        return {'data': self.data[index], 'label': self.labels[index]}

In [23]:
def get_image_tensor(file_path):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    
    image = Image.open(file_path)
    return transform(image)

def get_data_and_labels(directory_path, label):
    image_tensors, labels = [], []
    for file in os.listdir(directory_path):
        if file.endswith(".JPG"):
            file_path = directory_path + file
            image_tensor = get_image_tensor(file_path)
            image_tensors.append(image_tensor)
            labels.append(label)
            
    return image_tensors, labels
    
def get_data_sets(): 
    present_file_path = "data/MP152_ODOT009_REPELCAM/present/"
    not_present_file_path = "data/MP152_ODOT009_REPELCAM/not_present/"
    
    present_data, present_labels = get_data_and_labels(present_file_path, 1)
    not_present_data, not_present_labels = get_data_and_labels(not_present_file_path, 0)
    
    print("Number of wildlife present photos: " + str(len(present_data)))
    print("Number of no wildlife present photos: " + str(len(not_present_data)))
    
    data = present_data + not_present_data
    labels = present_labels + not_present_labels
    
    training_data, testing_data, training_labels, testing_labels = train_test_split(data, labels)
    
    print("\nNumber of training photos: " + str(len(training_data)))
    print("Number of testing photos: " + str(len(testing_data)))
    
    training_data_set = image_data_set(training_data, training_labels)
    testing_data_set = image_data_set(testing_data, testing_labels)
    
    return training_data_set, testing_data_set

def get_loaders(training_data_set, testing_data_set, batch_size):
    training_loader = torch.utils.data.DataLoader(dataset = training_data_set,
                                                  batch_size = batch_size,
                                                  shuffle = True)

    testing_loader = torch.utils.data.DataLoader(dataset = testing_data_set,
                                                 batch_size = batch_size,
                                                 shuffle = True)
    
    return training_loader, testing_loader

In [25]:
def train(model, training_loader, criterion, optimizer):
    model.train()
    running_loss = 0.0
    num_correct = 0
    for i, data in enumerate(training_loader):
        data, labels = data['data'].to(device), data['label'].to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, labels)
        running_loss += loss.item()
        _, predictions = torch.max(output.data, 1)
        num_correct += (predictions == labels).sum().item()
        loss.backward()
        optimizer.step()
    
    loss = running_loss/len(training_loader.dataset)
    accuracy = num_correct/len(training_loader.dataset)
    return loss, accuracy

def test(model, testing_loader, criterion):
    model.eval()
    running_loss = 0.0
    num_correct = 0
    for i, data in enumerate(testing_loader):
        data, labels = data['data'].to(device), data['label'].to(device)
        output = model(data)
        loss = criterion(output, labels)
        running_loss += loss.item()
        _, predictions = torch.max(output.data, 1)
        num_correct += (predictions == labels).sum().item()
    
    loss = running_loss/len(testing_loader.dataset)
    accuracy = num_correct/len(testing_loader.dataset)
    return loss, accuracy

In [26]:
def train_and_test(model, training_loader, testing_loader, device):
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    
    for epoch in range(5):
        print("epoch: " + str(epoch))
        
        training_loss, training_accuracy = train(model, training_loader, criterion, optimizer)
        print("training loss: " + str(training_loss) + " and training accuracy: " + str(training_accuracy))
        
        testing_loss, testing_accuracy = test(model, testing_loader, criterion)
        print("testing loss: " + str(testing_loss) + " and testing accuracy: " + str(testing_accuracy))

In [27]:
def train_and_test_VGG(training_loader, testing_loader, device, num_classes):
    print("\nTraining and Testing VGG")
    vgg11 = models.vgg11(weights = models.VGG11_Weights.DEFAULT)
    vgg11.classifier[6].out_features = num_classes
    train_and_test(vgg11, training_loader, testing_loader, device)

def train_and_test_ResNet(training_loader, testing_loader, device, num_classes):
    print("\nTraining and Testing ResNet")
    resnet50 = models.resnet50(weights = models.ResNet50_Weights.DEFAULT)
    resnet50.fc.out_features = num_classes
    train_and_test(resnet50, training_loader, testing_loader, device)
    
def train_and_test_AlexNet(training_loader, testing_loader, device, num_classes):
    print("\nTraining and Testing AlexNet")
    alexnet = models.alexnet(weights = models.AlexNet_Weights.DEFAULT)
    alexnet.classifier[6].out_features = num_classes
    train_and_test(alexnet, training_loader, testing_loader, device)
    
def train_and_test_Faster_RCNN(training_loader, testing_loader, device, num_classes):
    print("\nTraining and Testing Faster R-CNN")
    faster_rcnn = models.detection.fasterrcnn_resnet50_fpn_v2(weights = models.detection.FasterRCNN_ResNet50_FPN_V2_Weights.DEFAULT)
    faster_rcnn.roi_heads.box_predictor.bbox_pred.out_features = num_classes
    train_and_test(faster_rcnn, training_loader, testing_loader, device)
    
def train_and_test_YOLO(training_loader, testing_loader, device, num_classes):
    print("\nTraining and Testing YOLO")
    yolo = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
    train_and_test(yolo, training_loader, testing_loader, device)

#Orchestration

In [29]:
num_classes = 2
batch_size = 10

training_data_set, testing_data_set = get_data_sets()
training_loader, testing_loader = get_loaders(training_data_set, testing_data_set, batch_size)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()

Number of wildlife present photos: 223
Number of no wildlife present photos: 363

Number of training photos: 439
Number of testing photos: 147


In [30]:
train_and_test_VGG(training_loader, testing_loader, device, num_classes)


Training and Testing VGG


Downloading: "https://download.pytorch.org/models/vgg11-8a719046.pth" to /root/.cache/torch/hub/checkpoints/vgg11-8a719046.pth


  0%|          | 0.00/507M [00:00<?, ?B/s]

epoch: 0
training loss: 0.1032276546968686 and training accuracy: 0.7380410022779044
testing loss: 0.035089081748813186 and testing accuracy: 0.8979591836734694
epoch: 1
training loss: 0.03536020442504003 and training accuracy: 0.8747152619589977
testing loss: 0.037175028350482996 and testing accuracy: 0.8571428571428571
epoch: 2
training loss: 0.03026204425544565 and training accuracy: 0.8724373576309795
testing loss: 0.0360075586608478 and testing accuracy: 0.9047619047619048
epoch: 3
training loss: 0.03533155465570696 and training accuracy: 0.8633257403189066
testing loss: 0.026326827502169577 and testing accuracy: 0.9183673469387755
epoch: 4
training loss: 0.0225277856005046 and training accuracy: 0.9020501138952164
testing loss: 0.027270812148443697 and testing accuracy: 0.9047619047619048


In [31]:
train_and_test_ResNet(training_loader, testing_loader, device, num_classes)


Training and Testing ResNet


Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

epoch: 0
training loss: 0.31502008343068777 and training accuracy: 0.5261958997722096
testing loss: 0.07236364524380691 and testing accuracy: 0.8571428571428571
epoch: 1
training loss: 0.04303011674951584 and training accuracy: 0.8678815489749431
testing loss: 0.03625363968059319 and testing accuracy: 0.8979591836734694
epoch: 2
training loss: 0.02678535781919549 and training accuracy: 0.9157175398633257
testing loss: 0.03096347586030052 and testing accuracy: 0.891156462585034
epoch: 3
training loss: 0.02347590050454292 and training accuracy: 0.9111617312072893
testing loss: 0.02707627741303168 and testing accuracy: 0.9115646258503401
epoch: 4
training loss: 0.016334542656711946 and training accuracy: 0.9339407744874715
testing loss: 0.02328320577436564 and testing accuracy: 0.9319727891156463


In [32]:
train_and_test_AlexNet(training_loader, testing_loader, device, num_classes)


Training and Testing AlexNet


Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth


  0%|          | 0.00/233M [00:00<?, ?B/s]

epoch: 0
training loss: 0.12469446363780384 and training accuracy: 0.7129840546697038
testing loss: 0.06630543411588993 and testing accuracy: 0.8299319727891157
epoch: 1
training loss: 0.03815333075091616 and training accuracy: 0.8496583143507973
testing loss: 0.03716475795320913 and testing accuracy: 0.8979591836734694
epoch: 2
training loss: 0.03214289867457062 and training accuracy: 0.8724373576309795
testing loss: 0.03810355785701956 and testing accuracy: 0.8843537414965986
epoch: 3
training loss: 0.03092758581975734 and training accuracy: 0.8838268792710706
testing loss: 0.03693806760165156 and testing accuracy: 0.8775510204081632
epoch: 4
training loss: 0.027654808574697695 and training accuracy: 0.8952164009111617
testing loss: 0.03637052722731415 and testing accuracy: 0.8775510204081632


In [33]:
#TODO: Get both of these working
#train_and_test_Faster_RCNN(training_loader, testing_loader, device, num_classes)
#train_and_test_YOLO(training_loader, testing_loader, device, num_classes)